In [14]:
from numpy import pi, cos, sin, arccos, sqrt, log10

def grados_a_radianes(g, m, s, sgn):
    '''
    Recibo grados, minutos y segundos y devuelve los radianes correspondientes
    Por convenciónd de la DG 61: el signo significa norte/este (-1) o sur/oeste(1)
    '''

    #Grados expresados en decimales
    gd = (g + (m+s/60)/60)*sgn
    rad = gd*pi/180
    return rad

def decimales_a_sexagesimales(dec):
    g = int(dec)
    m = int((dec - int(dec))*60)
    s = (((dec - int(dec))*60) - m)*60

    s = str(g) + "° " + str(m) + "' " + str(s) + " ''"
    return s

lat_A = grados_a_radianes(34, 30, 5, -1)
lon_A = grados_a_radianes(60, 30, 2.1, -1)
lat_B = grados_a_radianes(34, 35, 14, -1)
lon_B = grados_a_radianes(60, 27, 53.14, -1)

print(lat_A*180/pi)
print(lon_A*180/pi)
print(lat_B*180/pi)
print(lon_B*180/pi)

-34.50138888888889
-60.50058333333333
-34.587222222222216
-60.4647611111111


### Distancia

In [15]:
def distancia(lat_A, lon_A, lat_B, lon_B):
    '''
    A partir de las coordenadas (en radianes) de dos puntos de la Tierra, calcula la distancia (en km) según la formula de la DG 61
    '''
    d = arccos(cos(lat_A)*cos(lat_B)*cos(lon_A-lon_B)+sin(lat_A)*sin(lat_B))
    d = d*180/pi #paso el arcoseno en radianes a grados
    d = 111.194*d
    return d
    

L = distancia(lat_A, lon_A, lat_B, lon_B)
print(str(L) + " km")

10.092339134866082 km


### Acimut

In [16]:
def acimut_O(lat_A, lon_A, lat_B, lon_B):
    '''
    Acimut (orientación de la antena) de la estación más occidental
    '''
    t = cos(lat_B)*sin(lon_B-lon_A)
    u = cos(lat_B)*cos(lon_B-lon_A)*sin(lat_A) - cos(lat_A)*sin(lat_B)
    
    #Argumento del arcocoseno
    ac = (cos(lat_B)*cos(lon_B-lon_A)*sin(lat_A) - cos(lat_A)*sin(lat_B))/sqrt(t**2 + u**2)
    
    #Hago arcocoseno y paso a sexagesimales
    ac = (180/pi)*arccos(ac) 

    return ac


def acimut_E(lat_A, lon_A, lat_B, lon_B):
    '''
    Acimut (orientación de la antena) de la estación más oriental
    '''

    v = cos(lat_A)*sin(lon_A-lon_B)
    u = cos(lat_A)*cos(lon_A-lon_B)*sin(lat_B) - cos(lat_B)*sin(lat_A)

    #Argumento del arcocoseno
    ac = (cos(lat_B)*sin(lat_A) - cos(lat_A)*cos(lon_A-lon_B)*sin(lat_B))/sqrt(v**2 + u**2)

    #Hago arcocoseno, paso a sexagesimales y termino la expresión
    ac = 180 + (180/pi)*arccos(ac)

    return ac

print(acimut_O(lat_A, lon_A, lat_B, lon_B))
print(decimales_a_sexagesimales(acimut_O(lat_A, lon_A, lat_B, lon_B)))

print(acimut_E(lat_A, lon_A, lat_B, lon_B))
print(decimales_a_sexagesimales(acimut_E(lat_A, lon_A, lat_B, lon_B)))

18.960924987246457
18° 57' 39.329954087246506 ''
198.98123774614348
198° 58' 52.455886116520105 ''


### Atenuación espacio libre

In [17]:
def atenuacion_EL(f, d):
    '''
    Calcula la atenuación (en dB) de espacio libre en función de la frecuencia [MHz] y la distancia en MHz
    '''
    at = 32.44 + 20*log10(f) + 20*log10(d)
    
    return at


f = 7296 #[MHz]
print(str(atenuacion_EL(f, L)) + " dB")

129.7815332218877 dB


### Margen de desvanecimiento

In [18]:
def barnett_vigants(d, f, A, B, R):
    '''
    Calcula el margen de desvanecimiento (en dB) necesario para tener la confiabilidad R para un enlace con distancia d [km], frecuencia f [GHz]
    y factores A (aspereza terreno) y B (anualización del peor mes)
    '''
    fm = 30*log10(d) + 10*log10(6*A*B*f) - 10*log10(1-R) - 70
    return fm

def confiabilidad(d, f, A, B, fm):

    R = fm - 30*log10(d) - 10*log10(6*A*B*f) + 70
    R = 10**(R/(-10))
    R = 1 - R 
    return R

A = 1
B = 0.25
R = 0.999975



print(str(barnett_vigants(L, f, A, B, R)) + " dB")

# fm = 31.3

# print(str(confiabilidad(L, f, A, B, fm)*100) + "%")

46.53211583025167 dB
